In [ ]:
#############################################################################################################
## Estimate monthly ozone values based on X,Y point and date information (month)
## Coded specifically towards data being provided by researcher, MK
## Can easily be adapted to other input data. Changes would probably be need to made to the date logic
## 
## Currently configured to run cell-by-cell in a Notebook. Should be configured properly with functions and main

In [ ]:
import arcpy, os
from datetime import datetime

arcpy.env.workspace = r'in_memory'
arcpy.env.overwriteOutput = True

In [ ]:
## Data should be copied to the same directory that this script is being stored
path = os.path.abspath(os.getcwd())

## Monthly Total Ozone Means data was pulled from woudc.org and joined to the station points to create this layer
monthly_ozone_points_layer =  path + r'\Data.gdb\MonthlyTotalOzone_Points'

## Data Inputs
observations_layer = input("Enter Observation Feature Class (Full Path): ")
output_layer_location = input("Enter Directory Location for Output Feature Class (FGDB Required): ")


proc_layer_name = r'proc_layer_20191213_ozone'
procname = "proc_layer__" + str(datetime.now().strftime("%Y%m%d_%H%M")) + "_ozone"
arcpy.env.outputCoordinateSystem = arcpy.Describe(observations_layer).spatialReference


## Local Variables
count = 0
results = []
kriging_result = None

In [ ]:
print(observations_layer)

## Copy ozone and observations layer to in_memory workspace
observations = arcpy.CopyFeatures_management(observations_layer, "observations")
monthly_ozone_points = arcpy.CopyFeatures_management(monthly_ozone_points_layer, "monthly_ozone_points")

In [ ]:
if not arcpy.ListFields(observations, "FromRaster"):
    arcpy.AddField_management(observations,'FromRaster', "SHORT")
    
if not arcpy.ListFields(observations, "StationCount"):
    arcpy.AddField_management(observations,'StationCount', "SHORT")    

In [ ]:
## Extent representing the area of research. This is to limit the processing time for raster creation.

desc = arcpy.Describe(monthly_ozone_points_layer)

xmin = desc.extent.XMin
xmax = desc.extent.XMax
ymin = desc.extent.YMin
ymax = desc.extent.YMax

# print (xmin, ymin, xmax, ymax)

arcpy.env.extent = arcpy.Extent(xmin, ymin, xmax, ymax)

In [ ]:
## Lookup to convert full month to month abbreviation
monthLookup = {'January':'Jan','February':'Feb','March':'Mar','April':'Apr','May':'May','June':'Jun','July':'Jul','August':'Aug','September':'Sep','October':'Oct','November':'Nov','December':'Dec',}

In [ ]:
obs_fields = [field.name for field in arcpy.ListFields(observations)]
proc_layer = None
from_raster = 0

## Loop through observations that contain location data (Lat/Lon) and Year
for cnt, row in enumerate(arcpy.da.SearchCursor(observations,where_clause="Latitude is not NULL and Year_ is not null",
                            field_names=obs_fields)):

## Select row (observation) and copy to in_memory layer
    where = "UnitID = '" + row[obs_fields.index('UnitID')] + "'"
    observation_layer = arcpy.management.SelectLayerByAttribute(observations, "NEW_SELECTION", where, None)    
    observation = arcpy.CopyFeatures_management(observation_layer, "observation")

    if not row[obs_fields.index('Month_')]:
        continue

    if not row[obs_fields.index('Month_')].strip() in monthLookup.keys():
        print("Key Error, Invalid Month: " + row[obs_fields.index('Month_')])
        continue
    
    month = monthLookup[row[obs_fields.index('Month_')].strip()]
    month_field = 'MonthlyTotalOzone_'+month
    year = row[obs_fields.index('Year_')]
    
## Select Ozone stations with data for observation Month/Year    
    where = "MonthlyTotalOzone_Year = " + str(year) + ' AND MonthlyTotalOzone_' + month + ' > 0'
    print(row[obs_fields.index('UnitID')], where)
    selection = arcpy.management.SelectLayerByAttribute(monthly_ozone_points, "NEW_SELECTION", where, None)
    
    station_count = int(arcpy.GetCount_management(selection).getOutput(0))
    update_fields = ['StationCount','FromRaster']
    update_vals = [station_count]
    
## If there are stations with data, do process    
    if station_count > 0:
        arcpy.Near_analysis(in_features=observation, near_features=selection, search_radius="", location="NO_LOCATION", angle="NO_ANGLE", method="GEODESIC")

## If Kriging runs successfully, get value from Kriging result
        try:
            kriging_result = arcpy.sa.Kriging(selection, month_field, "Spherical 100000 # # #", 100000, "VARIABLE 12", None); kriging_result.save("kriging_result" + str(cnt))
            update_vals.append(1)
            raster_list = [[kriging_result, "average_uv"]]
            arcpy.gp.ExtractMultiValuesToPoints_sa(observation, raster_list, "BILINEAR")
            
## If Kriging fails, assume too few station points, and populate with value from nearest station.
## ***Check processing log to be sure that processing errors are relatedt to input data and not other factors
        except Exception as err:
            print("Kriging Error: " + str(err))
            kriging_result = False
            update_vals.append(0)
            near_fid = [row[0] for row in arcpy.da.SearchCursor(observation, field_names=['NEAR_FID'])][0]
            avg_uv = [row[0] for row in arcpy.da.SearchCursor(monthly_ozone_points, field_names=[month_field], where_clause='OBJECTID='+str(near_fid))][0]
            arcpy.AddField_management(observation,'average_uv', "SINGLE")
            update_fields.append('average_uv')
            update_vals.append(avg_uv)
## No stations with data, populate attributes with blank values    
    else:
        arcpy.AddField_management(observation,'NEAR_FID', "INTEGER")
        arcpy.AddField_management(observation,'NEAR_DIST', "DOUBLE")        
        arcpy.AddField_management(observation,'average_uv', "SINGLE")
        update_vals.append(0)
        update_fields.append('NEAR_FID')  
        update_fields.append('NEAR_DIST')         
        update_fields.append('average_uv')   
        update_vals.append(0)
        update_vals.append(0)
        update_vals.append(0)
        
              
## Update observation with station_count and from_raster values. More if any failures.
    with arcpy.da.UpdateCursor(observation, field_names=update_fields) as update:
        for row in update:
            for i in range(len(update_fields)):
                row[i] = update_vals[i]
                update.updateRow(row) 

## On first run, create/overwrite proc_layer
    if not arcpy.Exists(proc_layer_name):
        proc_layer = arcpy.CreateFeatureclass_management("in_memory", proc_layer_name, template=observation)
        print("Create proc_layer")


## Append observation to proc_layer        
    arcpy.Append_management(observation, proc_layer)


In [ ]:
## Copy in_memory proc_layer to proc_layer in output directory
arcpy.CopyFeatures_management(proc_layer, output_layer_location + "proc_layer")